# 1.. Installation de PySpark


In [ ]:
!pip install pyspark


# *2. Créer un DataFrame à partir du fichier ngram.csv*

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [3]:
spark = SparkSession.builder.appName("NGramAnalysis").getOrCreate()

In [4]:
# Définir un schéma avec les noms de colonnes et leurs types de données
schema = StructType([
    StructField("ngram", StringType(), True),  # Bi-gramme lui-même
    StructField("Year", IntegerType(), True),  # Année d'apparition du bi-gramme
    StructField("Count", IntegerType(), True),  # Nombre de fois où le bi-gramme est apparu
    StructField("Pages", IntegerType(), True),  # Nombre de pages où le bi-gramme est apparu
    StructField("Books", IntegerType(), True)   # Nombre de livres distincts contenant le bi-gramme
])

In [5]:
!wget http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-us-all-4gram-20090715-269.csv.zip -O ngram.csv.zip

--2024-11-12 19:35:24--  http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-us-all-4gram-20090715-269.csv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.207, 74.125.26.207, 172.217.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142740409 (136M) [application/zip]
Saving to: ‘ngram.csv.zip’

ngram.csv.zip       100%[===================>] 136.13M   127MB/s    in 1.1s    

2024-11-12 19:35:25 (127 MB/s) - ‘ngram.csv.zip’ saved [142740409/142740409]



In [6]:
!ls -lh ngram.csv.zip


-rw-r--r-- 1 root root 137M Dec 23  2010 ngram.csv.zip


In [7]:
#  Unzip the file
!unzip /content/ngram.csv.zip -d /content/ngram_extracted

Archive:  /content/ngram.csv.zip
  inflating: /content/ngram_extracted/googlebooks-eng-us-all-4gram-20090715-269.csv  


In [8]:
#  Verify the extraction
!ls /content/ngram_extracted

googlebooks-eng-us-all-4gram-20090715-269.csv


In [10]:
df = spark.read.csv("/content/ngram_extracted/*.csv", header=False, schema=schema, sep="\t", encoding="UTF-8")


In [11]:
df.show(5)

+---------------+----+-----+-----+-----+
|          ngram|Year|Count|Pages|Books|
+---------------+----+-----+-----+-----+
|"! "" "" Jacob"|1847|    1|    1|    1|
|"! "" "" Jacob"|1856|    3|    3|    3|
|"! "" "" Jacob"|1859|    2|    2|    2|
|"! "" "" Jacob"|1867|    1|    1|    1|
|"! "" "" Jacob"|1871|    1|    1|    1|
+---------------+----+-----+-----+-----+
only showing top 5 rows



In [12]:
df.printSchema()

root
 |-- ngram: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Pages: integer (nullable = true)
 |-- Books: integer (nullable = true)



*3. Enregistrer le DataFrame en tant que table temporaire*

In [13]:
df.createOrReplaceTempView("ngrams")


# *4. Répondre aux requêtes*

### (1) Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq

In [14]:
#SQL
spark.sql("SELECT * FROM ngrams WHERE Count > 5").show()

#API Spark
df.filter(df["Count"] > 5).show()


+----------------+----+-----+-----+-----+
|           ngram|Year|Count|Pages|Books|
+----------------+----+-----+-----+-----+
| "! "" "" Jacob"|1941|    6|    6|    4|
|"! "" "" Others"|1955|    6|    6|    6|
|"! "" "" Others"|1968|   16|   16|   16|
|"! "" "" Others"|1983|    6|    6|    6|
|"! "" "" Others"|1993|   10|    6|    6|
|"! "" "" Others"|1996|    6|    6|    6|
|"! "" "" Partly"|1900|    6|    6|    6|
|     "! "" ( 30"|1992|    6|    6|    6|
|     "! "" ( 30"|1993|    6|    6|    6|
|     "! "" ( 30"|1995|    9|    9|    9|
|     "! "" ( 30"|1996|    6|    6|    5|
|     "! "" ( 30"|1997|    8|    8|    8|
|     "! "" ( 30"|1998|   10|    9|    7|
|     "! "" ( 30"|2000|   12|   11|    8|
|     "! "" ( 30"|2002|   10|    9|    8|
|     "! "" ( 30"|2004|   11|   11|   10|
|     "! "" ( 30"|2005|   11|   10|    8|
|     "! "" ( 30"|2006|    9|    9|    9|
|     "! "" ( 30"|2007|   15|   15|   12|
|   "! "" ( Come"|1944|    8|    8|    8|
+----------------+----+-----+-----

### (2) Retourner le nombre total de bi-grammes dans chaque année


In [ ]:
#SQL
spark.sql("SELECT Year, COUNT(ngram) AS total_bigrams FROM ngrams GROUP BY Year").show()
#API Spark
df.groupBy("Year").count().show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1959|       192841|
|1829|        32044|
|1896|       130771|
|1990|       276657|
|1903|       142959|
|1975|       247752|
|1884|       113475|
|1977|       249836|
|1888|       118889|
|1787|          591|
|1766|           22|
|1808|        10546|
|1743|          136|
|1863|        54204|
|1924|       136632|
|2003|       336315|
|1823|        19295|
|1745|            6|
|2007|       326312|
|1869|        85276|
+----+-------------+
only showing top 20 rows



### (3) Retourner les bi-grammes qui ont le plus grand nombre de Count dans chaque année


In [ ]:
#SQL
spark.sql("""
SELECT Year, ngram, MAX(Count) AS max_count
FROM ngrams
GROUP BY Year, ngram
ORDER BY Year ASC
""").show()

#API sparkSQL
from pyspark.sql import functions as F

df.groupBy("Year", "ngram").agg(F.max("Count").alias("max_count")).orderBy("Year").show()


+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
|1834| ! 09|        4|
|1836| ! 09|        1|
|1839| ! 09|        1|
|1840| ! 09|        1|
|1841| ! 09|        2|
|1845| ! 09|        1|
|1847| ! 09|        2|
|1848| ! 09|        1|
|1856| ! 09|        1|
|1857| ! 09|        2|
+----+-----+---------+
only showing top 20 rows

+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
|1834| ! 09|        4|
|1836| ! 09|        1|
|1839| ! 09|        1|
|1840| ! 09|        1|
|1841| ! 09|        2|
|1845| !

### (4) Retourner tous les bi-grammes qui sont apparus dans 20 années différentes


In [ ]:
#SQL
spark.sql("""
SELECT ngram
FROM ngrams
GROUP BY ngram
HAVING COUNT(DISTINCT Year) = 20
""").show()
#API sparkSQL
df.groupBy("ngram").agg(F.countDistinct("Year").alias("years_count")).filter("years_count = 20").show()


+-----+
|ngram|
+-----+
+-----+

+-----+-----------+
|ngram|years_count|
+-----+-----------+
+-----+-----------+



### (5) Retourner tous les bi-grammes qui contiennent ! dans la première partie et 9 dans la deuxième partie


In [ ]:
#SQL
spark.sql("""
SELECT *
FROM ngrams
WHERE ngram LIKE '! % 9'
""").show()

#API sparkSQL
df.filter(df["ngram"].like("!% 9")).show()


### (6) Retourner les bi-grammes qui sont apparus dans toutes les années présentes dans les données


In [ ]:
#SQL
total_years = spark.sql("SELECT COUNT(DISTINCT Year) as total_years FROM ngrams").collect()[0]["total_years"]
spark.sql(f"""
SELECT ngram
FROM ngrams
GROUP BY ngram
HAVING COUNT(DISTINCT Year) = {total_years}
""").show()

#API sparkSQL
total_years = df.select(F.countDistinct("Year")).collect()[0][0]

df.groupBy("ngram").agg(F.countDistinct("Year").alias("year_count")).filter(f"year_count = {total_years}").show()


### (7) Retourner le nombre total de pages et de livres dans lesquels chaque bi-gramme apparaît, trié par ordre alphabétique


In [ ]:
#SQL
spark.sql("""
SELECT ngram, SUM(Pages) AS total_pages, SUM(Books) AS total_books
FROM ngrams
GROUP BY ngram
ORDER BY ngram ASC
""").show()

#API sparkSQL
df.groupBy("ngram").agg(F.sum("Pages").alias("total_pages"), F.sum("Books").alias("total_books")).orderBy("ngram").show()


### (8) Retourner le nombre total de bi-grammes différents dans chaque année, trié par ordre décroissant de l'année


In [ ]:
#SQL
spark.sql("""
SELECT Year, COUNT(DISTINCT ngram) AS total_bigrams
FROM ngrams
GROUP BY Year
ORDER BY Year DESC
""").show()

#API sparkSQL
df.groupBy("Year").agg(F.countDistinct("ngram").alias("total_bigrams")).orderBy(F.desc("Year")).show()
